In [2]:
from graph import InferenceWorker
from bs4 import BeautifulSoup
import requests
import feedparser

In [4]:
class RSSWorker(InferenceWorker):
    def __call__(self, t, d):
        if t == "link":
            parsed_feed = feedparser.parse(d)
            yield "text", self.text_from_rss(parsed_feed), "parse"
        else:
            yield None

    def text_from_rss(self, parsed_feed):
        permalinks = []
        for entry in parsed_feed.entries:
            permalinks.append(entry.link)
        all_texts = []
        for i in range(len(permalinks)): 
            html = requests.get(permalinks[i]).content.decode("utf-8")
            soup = BeautifulSoup(html, 'html.parser')
            texts = soup.findAll(text=True)
            good_tags = ['p']
            text = u" ".join(t.strip() for t in texts if good_tags.count(t.parent.name) > 0)
            all_texts.append(text)
        return all_texts

In [6]:
worker = RSSWorker()
worker.serialize()

Worker saved!


In [8]:
for x in worker("link", "https://meduza.io/rss/news"):
    print(x)
    print(len(x[1]))

('text', ['Данное сообщение (материал) создано и\xa0(или) распространено иностранным средством массовой информации, выполняющим функции иностранного агента, и\xa0(или) российским юридическим лицом, выполняющим функции иностранного агента. Нам нужна ваша помощь. Пожалуйста,   . «Мы\xa0находимся во\xa0[временном] окне, когда вторжение может начаться в\xa0любое время, в\xa0том числе во\xa0время Олимпийских игр»,\xa0— госсекретарь США Энтони Блинкен. Советник президента США по\xa0национальной безопасности Джейк Салливан , что Россия сконцентрировала у\xa0границ Украины достаточно войск для вторжения, но\xa0у\xa0США нет информации, принял\xa0ли президент России Владимир Путин окончательное решение о\xa0том, что вторжение будет. Если Россия все-таки нападет на\xa0Украину, вторжение может начаться с\xa0воздушных бомбардировок и\xa0ракетных атак, сказал Салливан. Он\xa0также допустил, что российские войска могут атаковать Киев. , источники Politico. По Der Spiegel, эту дату правительству США и